In [1]:
from com.xilinx.rapidwright.design import Design
from com.xilinx.rapidwright.design import DesignTools
from com.xilinx.rapidwright.design import NetType
from com.xilinx.rapidwright.design import Unisim
from com.xilinx.rapidwright.design import Module
from com.xilinx.rapidwright.design import SitePinInst
from com.xilinx.rapidwright.device import Device
from com.xilinx.rapidwright.device import Site
from com.xilinx.rapidwright.device import BEL
from com.xilinx.rapidwright.device import SitePIP
from com.xilinx.rapidwright.device import SiteTypeEnum
from com.xilinx.rapidwright.design import PinType 
from com.xilinx.rapidwright.design import Net 
from com.xilinx.rapidwright.design import NetType
from com.xilinx.rapidwright.design.tools import LUTTools
from com.xilinx.rapidwright.edif import EDIFCell
from com.xilinx.rapidwright.edif import EDIFCellInst
from com.xilinx.rapidwright.edif import EDIFDirection
from com.xilinx.rapidwright.edif import EDIFNet
from com.xilinx.rapidwright.edif import EDIFNetlist
from com.xilinx.rapidwright.edif import EDIFPort
from com.xilinx.rapidwright.edif import EDIFPortInst
from com.xilinx.rapidwright.edif import EDIFTools
from com.xilinx.rapidwright.edif import EDIFParser
from com.xilinx.rapidwright.edif import EDIFLibrary
from com.xilinx.rapidwright.edif import EDIFPropertyObject
from com.xilinx.rapidwright.edif import EDIFValueType
from com.xilinx.rapidwright.util import FileTools
from com.xilinx.rapidwright.router import RouteNode
from com.xilinx.rapidwright.router import Router
from com.xilinx.rapidwright.util   import MessageGenerator

### JSONReader.py

In [2]:
import json
import os

def read_file(filename):
    with open(filename, 'r') as file:
        jnet = json.load(file)
    return jnet



def buildLibrary(jnet, library):
    topModule = library.getNetlist().getTopCell().getName()
    for cell in jnet['modules']:
        if cell == topModule: continue
        if library.containsCell(cell): continue
        ret = EDIFCell(library,cell)
        for port in jnet['modules'][cell]['ports'].keys():
            if jnet['modules'][cell]['ports'][port]['direction'] == "input":
                direction = EDIFDirection.INPUT
            elif jnet['modules'][cell]['ports'][port]['direction'] == "output":
                direction = EDIFDirection.OUTPUT
            else:
                direction = EDIFDirection.INOUT
            ret.createPort(port,direction,1)
        library.addCell(ret)


def defineExternalPorts(jnet, topCell):
    topModule = topCell.getName()
    for port in jnet['modules'][topModule]["ports"]:
        port_data = jnet['modules'][topModule]["ports"].get(port)
        port_name = str(port)
        num_pins = len(port_data.get("bits"))
        for i in range(num_pins):
            name = port_name + ("" if num_pins == 1 else ("[" + str(i)  + "]"))
            connectionID = port_data.get("bits")[i]
            direction = port_data.get("direction")
            if direction == "input":
                ret = topCell.createPort(name,EDIFDirection.INPUT, 1)
            elif direction == "output":
                ret = topCell.createPort(name,EDIFDirection.OUTPUT, 1)
            else:
                ret = topCell.createPort(name,EDIFDirection.INOUT, 1)

                

def createEDIFCellInsts(jnet, topCell, library):
    topModule = topCell.getName()
    for cell in jnet['modules'][topModule]["cells"]:
        cell_data = jnet['modules']['top']['cells'].get(cell)
        cell_name = cell
        cell_type = cell_data['type']
        cell_properties = cell_data.get('parameters')
        cell_ports = cell_data['port_directions'].keys()
        cell_port_directions = cell_data['port_directions'].values()
        cell_ports_connectionID = cell_data['connections'].values()
        ret = EDIFCellInst(cell_name, library.getCell(cell_type),topCell)
        for proprty in cell_properties.keys():
            value = cell_properties.get(proprty)
            if value == 'x': continue
            ret.addProperty(proprty,str(int(value,2)), EDIFValueType.INTEGER )


def createEDIFPorts(jnet, topCell):
    topModule = topCell.getName()
    ports = []
    for net in jnet['modules'][topModule]["netnames"]:
        connectionIDs = jnet['modules'][topModule]["netnames"][net]['bits']
        for i in range(len(connectionIDs)):
            ret = topCell.createNet(str(net) + ("" if (len(connectionIDs)) == 1 else ("[" + str(i)  + "]")))
            for cell in jnet['modules'][topModule]["cells"]:
                for port in jnet['modules'][topModule]["cells"][cell]['connections']:
                    value = jnet['modules'][topModule]["cells"][cell]['connections'].get(port)
                    if value[0] == connectionIDs[i]:
                        if ((port,cell) in ports): continue
                        else:
                            ret.createPortInst(port,topCell.getCellInst(cell))
                            ports.append((port,cell))
            for port in jnet['modules'][topModule]['ports']:
                topConnectionIDs = jnet['modules'][topModule]['ports'][port]['bits']
                for j in range(len(topConnectionIDs)):
                    if topConnectionIDs[j] == connectionIDs[i]:
                        name = str(port) + ("" if (len(topConnectionIDs)) == 1 else ("[" + str(j)  + "]"))
                        if ((name,topModule) in ports): continue
                        else:
                            ret.createPortInst(topCell.getPort(name))
                            ports.append((name,topModule))

                            
def cleanEmptyNets(topCell):
    nets = []
    for net in topCell.getNets():
        if not (net.getPortInsts()):
            nets.append(str(net))
    for i in nets:
        topCell.removeNet(i)

        

def createStaticSourceNets(jnet, topCell, netlist):
    topModule = topCell.getName()
    gnd = EDIFTools.getStaticNet(NetType.GND, topCell, netlist);
    vcc = EDIFTools.getStaticNet(NetType.VCC, topCell, netlist);
    for cell in jnet['modules'][topModule]["cells"]:
        for port in jnet['modules'][topModule]["cells"][cell]['connections']:
            value = jnet['modules'][topModule]["cells"][cell]['connections'].get(port)
            if value[0] == "0":
                gnd.createPortInst(port , topCell.getCellInst(cell))
            elif value[0] == "1":
                vcc.createPortInst(port , topCell.getCellInst(cell))


def read_json (filename, topModule, device):
    jnet = read_file(filename)
    design = Design(topModule,device)
    netlist = design.getNetlist()
    topCell = netlist.getTopCell()
    library = netlist.getLibrary("hdi_primitives")
    buildLibrary(jnet, library)
    defineExternalPorts(jnet, topCell)    
    createEDIFCellInsts(jnet, topCell, library)
    createEDIFPorts(jnet, topCell)
    cleanEmptyNets(topCell)
    createStaticSourceNets(jnet, topCell,netlist)
    return design



### ConstrainDesign.py

In [3]:
def placeIOBuffers(design , constraints):
    for pin in constraints.keys():
        for topnets in design.getNetlist().getTopCell().getNets().toArray().tolist():
            if pin == str(topnets):
                for cellName in design.getNetlist().getTopCell().getCellInsts().toArray().tolist():
                    if ("IBUF" in str(cellName.getCellType())) or ("OBUF" in str(cellName.getCellType())):
                        for entry in topnets.getPortInstMap():
                            if str(cellName) in str(entry):
                                ret = design.placeIOB(cellName, constraints.get(pin).get('LOC'), constraints.get(pin).get('IOSTANDARD'))
                                ret.addProperty("IOSTANDARD",constraints.get(pin).get("IOSTANDARD"))
                                ret.addProperty("SLEW",constraints.get(pin).get("SLEW"))
                                ret.addProperty("IN_TERM",constraints.get(pin).get("IN_TERM"))
                                ret.addProperty("PULL_TYPE",constraints.get(pin).get("PULL_TYPE"))

### Logical2PysicalCells.py

In [4]:
def placeCells(design):
    cell_Names = design.getNetlist().getTopCell().getCellInsts().toArray().tolist()
    placement_list = []
    for cn in cell_Names:
        if ("GND" in str(cn.getCellType())) or ("VCC" in str(cn.getCellType())):
             continue

        elif ("IBUF" in str(cn.getCellType())) or ("OBUF" in str(cn.getCellType())):
            continue

        elif ("BUFGCTRL" in str(cn.getCellType())):
            design.getNetlist().getTopCell().removeCellInst(str(cn))
            loc = "BUFGCTRL_X0Y16/BUFGCTRL"
            ret = design.createAndPlaceCell(str(cn),Unisim.valueOf(str(cn.getCellType())),loc) 
            ret.setProperties(cn.getProperties())
            ret.connectStaticSourceToPin(NetType.VCC ,"CE0")
            ret.connectStaticSourceToPin(NetType.VCC ,"S0")
            ret.connectStaticSourceToPin(NetType.GND ,"CE1")
            ret.connectStaticSourceToPin(NetType.GND ,"S1")
            ret.connectStaticSourceToPin(NetType.GND ,"IGNORE1")
            ret.connectStaticSourceToPin(NetType.GND ,"IGNORE0")


        elif ("FDRE" in str(cn.getCellType())): 
            bels = Design().createCell(str(cn),cn).getCompatiblePlacements().get(SiteTypeEnum.SLICEL)
            sites = design.getDevice().getAllCompatibleSites(SiteTypeEnum.SLICEL)    
            running = 1
            while running:
                bel_name = list(bels)[random.randint(0,len(bels)-1)]
                site = sites[random.randint(0,len(sites)-1)]
                bel = site.getBEL(bel_name)
                if (str(site)+"/"+str(bel)) not in placement_list:
                    placement_list.append((str(site)+"/"+str(bel))) 
                    running = 0
            loc = str(site)+"/"+str(bel.getName())
            unsm = Unisim.FDRE
            cell = str(cn)
            design.getNetlist().getTopCell().removeCellInst(str(cn))
            ret = design.createAndPlaceCell(cell,unsm,loc)        
            ret.connectStaticSourceToPin(NetType.GND ,"R")
            ret.connectStaticSourceToPin(NetType.VCC ,"CE")

        elif ("LUT" in str(cn.getCellType())):
            cell = design.createCell(str(cn),cn)
            bels = cell.getCompatiblePlacements().get(SiteTypeEnum.SLICEL)
            sites = design.getDevice().getAllCompatibleSites(SiteTypeEnum.SLICEL)    
            running = 1
            while running:
                bel_name = list(bels)[random.randint(0,len(bels)-1)]
                site = sites[random.randint(0,len(sites)-1)]
                bel = site.getBEL(bel_name)
                if (str(site)+"/"+str(bel)) not in placement_list:
                    placement_list.append((str(site)+"/"+str(bel))) 
                    running = 0
                    design.placeCell(cell,site,bel)
print ""

### Logical2PhysicalNets.py

In [5]:
def createNets(design):
    for i in design.getNets().toArray().tolist():
        design.removeNet(i)
    for i in design.getNetlist().getTopCell().getNets().toArray().tolist():
        if str(i) in design.getNetlist().getTopCell().getPortMap().keys() and not(str(i) == "clk"): continue  
        if str(i) == "VCC_NET": continue     
        if str(i) == "GND_NET": continue   
        design.createNet(i)
    
    
def createNetPins(design):
    for physNet in design.getNets().toArray().tolist():
        if str(physNet) in design.getNetlist().getTopCell().getPortMap().keys() and not(str(physNet) == "clk"): continue
        edifNet = physNet.getLogicalNet()
        portInsts = edifNet.getPortInsts().toArray().tolist()
        for portInst in portInsts:
            portName = portInst.getName()
            portCell = portInst.getCellInst()
            portDir = portInst.getDirection()
            physCell = design.getCell(str(portCell))
            if portInst.isPrimitiveStaticSource(): continue
            siteInst = physCell.getSiteInst()
            siteWires = []
            ret = physCell.getSitePinFromPortInst(portInst,siteWires)            
            physNet.createPin(portInst.isOutput(),siteWires[len(siteWires)-1],siteInst)

### SiteRouter.py

In [6]:
def routeSitePIPs(design):
    for edifNet in design.getNetlist().getTopCell().getNets().toArray().tolist():
        if str(edifNet) in design.getNetlist().getTopCell().getPortMap().keys() and not(str(edifNet) == "clk"): continue
        if  (str(edifNet) == "GND_NET"): continue
        portInsts = edifNet.getPortInsts().toArray().tolist()
        for portInst in portInsts:
            if portInst.isPrimitiveStaticSource(): continue
            portCell = portInst.getCellInst()
            physCell = design.getCell(str(portCell))
            siteInst = physCell.getSiteInst()
            if "BUFGCTRL" in str(physCell.getType()):
                if (not("CE0" in str(portInst.getName()))) and (not("S0" in str(portInst.getName()))): continue
                siteWires = []
                ret = physCell.getSitePinFromPortInst(portInst,siteWires)
                bel = siteWires[0].split("_")[0] 
                belpin = ""
                for bp in siteInst.getSite().getBELPins(siteWires[1]).tolist():
                    if str(bel) in str(bp):
                        belpin = bp
                        break
                sitepip = siteInst.getSitePIP(belpin)
                siteInst.addSitePIP(sitepip)
                siteInst.routeSite()
            elif "FDRE" in str(physCell.getType()):
                if ("CE" == str(portInst.getName())):
                    belpin = siteInst.getSite().getBEL("CEUSEDMUX").getPin("1")
                    sitepip = siteInst.getSitePIP(belpin)
                    siteInst.addSitePIP(sitepip)
                    siteInst.routeSite()  
                elif ("R" == str(portInst.getName())):
                    belpin = siteInst.getSite().getBEL("SRUSEDMUX").getPin("0")
                    sitepip = siteInst.getSitePIP(belpin)
                    siteInst.addSitePIP(sitepip)
                    siteInst.routeSite()  
                elif ("C" == str(portInst.getName())):
                    belpin = siteInst.getSite().getBEL("CLKINV").getPin("CLK")
                    sitepip = siteInst.getSitePIP(belpin)
                    siteInst.addSitePIP(sitepip)
                    siteInst.routeSite()
                elif  ("Q" == str(portInst.getName())):
                    siteWires = []
                    ret = physCell.getSitePinFromPortInst(portInst,siteWires)
                    if len(siteWires)>1:
                        belpin = ""
                        for bp in siteInst.getSite().getBELPins(siteWires[0]).tolist():
                            if str(siteInst.getSite().getBELPins(siteWires[1]).tolist()[0]).split(".")[0] in str(bp):
                                belpin = bp
                                break
                        sitepip = siteInst.getSitePIP(belpin)
                        siteInst.addSitePIP(sitepip)
                        siteInst.routeSite()                
                elif  ("D" == str(portInst.getName())):
                    siteWires = []
                    ret = physCell.getSitePinFromPortInst(portInst,siteWires)
                    if len(siteWires)>1:
                        bel = siteWires[0].split("_")[0] 
                        belpin = ""
                        for bp in siteInst.getSite().getBELPins(siteWires[1]).tolist():
                            if str(bel) in str(bp):
                                belpin = bp
                                break
                        sitepip = siteInst.getSitePIP(belpin)
                        siteInst.addSitePIP(sitepip)
                        siteInst.routeSite()
            elif "LUT" in str(physCell.getType()):
                siteWires = []
                ret = physCell.getSitePinFromPortInst(portInst,siteWires)
                if len(siteWires)>1 and portInst.isOutput():
                    if ("IOB" in str(siteInst.getSite().getSiteTypeEnum())): continue
                    belpin = ""
                    for bp in siteInst.getSite().getBELPins(siteWires[0]).tolist():
                        if str(siteInst.getSite().getBELPins(siteWires[1]).tolist()[0]).split(".")[0] in str(bp):
                            belpin = bp
                            break
                    sitepip = siteInst.getSitePIP(belpin)
                    siteInst.addSitePIP(sitepip)
                    siteInst.routeSite()
print ""

### DesignClockRouter.py

In [7]:
from java.util import HashSet
from java.util import List
from pprint import pprint
import Queue
from Queue import PriorityQueue

def cost (s1, s2):
    return abs(s1.getRpmX() - s2.getRpmX()) + abs(s1.getRpmY() - s2.getRpmY())


def closestSite (s1, possible_sites):
    p = PriorityQueue()
    for s2 in possible_sites:
        p.put((cost(s1,s2),s2))
    return p.get()

def createBUFHs(design):
    topCell = design.getNetlist().getTopCell()
    if (design.getNet("clk")):
        sites = []
        pins = design.getNet("clk").getSinkPins()
        for pin in pins:
            pin_site = pin.getSite()
            bufhce_sites =  design.getDevice().getAllCompatibleSites(SiteTypeEnum.BUFHCE).tolist()
            sites.append(closestSite(pin_site,bufhce_sites)[1])

        for i in range(len(pins)):
            exists = -1
            for j in range(i):
                if design.getCell("bufhce_"+str(j)):
                    if str(design.getCell("bufhce_"+str(j)).getSite()) == str(sites[i]):
                        exists = j

            if exists == -1:
                loc = str(sites[i]) +"/BUFHCE"
                ret = design.createAndPlaceCell("bufhce_"+str(i),Unisim.BUFHCE,loc) 
                ret.connectStaticSourceToPin(NetType.VCC ,"CE")
                ret.getSiteInst().addSitePIP(ret.getSiteInst().getSitePIP(ret.getSiteInst().getBEL("CEINV").getPin("CE")))
                ret.getSiteInst().routeSite()
                net = design.createNet("clk" + str(i))
                design.getNet("clk").removePin(pins[i])
                design.getNet("clk").addPin(SitePinInst("I",ret.getSiteInst()))
                net.addPin(SitePinInst("O",ret.getSiteInst()))
                net.addPin(pins[i])
                topCell.getNet("clk"+str(i)).createPortInst("O",ret)
                topCell.getNet("clk"+str(i)).addPortInst(topCell.getNet("clk").getPortInsts().toArray().tolist()[0])
                topCell.getNet("clk").createPortInst("I",ret)
                topCell.getNet("clk").removePortInst(topCell.getNet("clk").getPortInsts().toArray().tolist()[0])
            else:
                net = design.getNet("clk" + str(exists))
                design.getNet("clk").removePin(pins[i])
                net.addPin(pins[i])
                topCell.getNet("clk"+str(exists)).addPortInst(topCell.getNet("clk").getPortInsts().toArray().tolist()[0])
                topCell.getNet("clk").removePortInst(topCell.getNet("clk").getPortInsts().toArray().tolist()[0])


def costFunction(curr, snk):
    return curr.getManhattanDistance(snk) + curr.getLevel()/8  

def routeClockNet(net, usedPIPs):
    path = []
    for sink in net.getPins():
        if sink.equals(net.getSource()): 
            continue
        q = RouteNode.getPriorityQueue()
        q.add(net.getSource().getRouteNode())
        path.extend(findRoute(q,sink.getRouteNode())) 
    path = list(dict.fromkeys(path))
    net.setPIPs(path)
    return


def findRoute(q, snk):
    visited = HashSet()
    src = q.poll()
    base_cost = costFunction(src,snk)
    q.add(src)
    counter = 0
    while(not q.isEmpty()):
        curr = q.poll()
        if(curr.equals(snk)):
            return curr.getPIPsBackToSource()
        visited.add(curr)


        for wire in curr.getConnections():
            nextNode = RouteNode(wire,curr)
            if visited.contains(nextNode): continue
            #if wire.isRouteThru(): continue
            curr_cost = costFunction(nextNode,snk)
            nextNode.setCost(curr_cost)
            q.add(nextNode)
    # Unroutable situation
    print "Route failed!"
    return []


def routeClocks(design):
    usedPIPs = []
    createBUFHs(design)
    for net in design.getNets():
        if "clk" in str(net):  
            routeClockNet(net,usedPIPs)

### DesignRouter.py

In [8]:
def designRouter(design):
    for net in design.getNets():
            if "clk" in str(net):  
                net.lockRouting()

    Router(design).routeDesign()

    for net in design.getNets():
        if "clk" in str(net):
            net.unlockRouting()

### FASMgenerator.py

In [23]:
import re
from com.xilinx.rapidwright.design.tools import LUTTools

def get_half_name(half, is_m):
    if (is_m):
        return "SLICEL_X1" if half else "SLICEM_X0"
    else:
        return "SLICEL_X1" if half else "SLICEL_X0"
    
def get_main_clock_nets(design):
    nets = []
    for net in design.getNets():
        pins = net.getPins()
        for pin in pins:
            bel_type = pin.getSite().getSiteTypeEnum().toString()
            if "BUFGCTRL" in bel_type and pin.isOutPin():
                if not net in nets:
                    nets.append(net)
    return nets

def get_secondary_clock_nets(design):
    nets = []
    for net in design.getNets():
        pins = net.getPins()
        for pin in pins:
            bel_type = pin.getSite().getSiteTypeEnum().toString()
            if "BUFHCE" in bel_type and pin.isOutPin():
                if not net in nets:
                    nets.append(net)
    return nets       
    
def get_used_logic_tiles(design):
    tiles = dict()
    for cell in design.getCells():
        if "IOB" in cell.getTile().getName(): continue
        if "CLK" in cell.getTile().getName(): continue
        half = cell.getSite().getInstanceX() % 2
        if not cell.getTile() in tiles.keys():
            sites_0 = []
            sites_1 = []
            if half:
                sites_1.append(cell.getSiteInst())
            else:
                sites_0.append(cell.getSiteInst())
            tiles[cell.getTile()] = [sites_0,sites_1]
        else:
            sites_0 = tiles[cell.getTile()][0]
            sites_1 = tiles[cell.getTile()][1]
            if half:
                if len(sites_1) == 0:
                     sites_1.append(cell.getSiteInst())
            else:
                if len(sites_0) == 0:
                     sites_0.append(cell.getSiteInst())
            tiles[cell.getTile()] = [sites_0,sites_1]
    return tiles

        
        
def write_luts_config(tiles, tile, lines):
    for half in range(2):
        if len(tiles[tile][half]) == 0: continue
        site = tiles[tile][half][0]
        is_mtile = "CLBLM" in tile.getName()
        is_slicem = is_mtile and (half == 0)
        for cell in site.getCells():
            if "LUT" in cell.getType():
                val = str("{0:b}".format(int(cell.getProperty("INIT").getValue())))
                len_val = len(val)
                for i in range(64-len_val):
                    val = "0" + val
                lut_size = LUTTools.getLUTSize("""64'd""" + str(cell.getProperty("INIT").getValue()))
                pin_map_str = cell.getPinMappingsL2P()
                pin_map = dict()
                for i in pin_map_str.keys():
                    pin = cell.getBEL().getPin(pin_map_str.get(i))
                    if pin.isInput(): 
                        index = re.sub('\D', '', pin.getName())
                        if not index:
                            index = "1"
                        pin_map[i] = int(index)-1
                pins = pin_map.values()
                min_pin = min(pins)
                init = ""
                for i in range(2**lut_size):
                    index = 0
                    count = 0
                    for j in pins:
                        index = index | (((i>>j) & 1)<< (j-min_pin))
                    init = val[63-index] + init
                line = tile.getName() + "." + get_half_name(half,is_mtile) + "." + cell.getBEL().getName()[0] + "LUT.INIT[63:0] = 64'b" + init
                lines.append(line)
    
 
 
def write_ff_config(tiles, tile, lines):
    for half in range(2):
        if len(tiles[tile][half]) == 0: continue
        site = tiles[tile][half][0]
        is_mtile = "CLBLM" in tile.getName()
        is_slicem = is_mtile and (half == 0)
        fdre_check = 0
        for cell in site.getCells():
            if not "FDRE" in cell.getType(): continue
            fdre_check = 1
            line = tile.getName() + "." + get_half_name(half,is_mtile) + "." + cell.getBEL().getName() + ".ZINI"
            lines.append(line)
            line = tile.getName() + "." + get_half_name(half,is_mtile) + "." + cell.getBEL().getName() + ".ZRST"
            lines.append(line)
        if fdre_check:
            line = tile.getName() + "." + get_half_name(half,is_mtile) + ".FFSYNC"
            lines.append(line)
            line = tile.getName() + "." + get_half_name(half,is_mtile) + ".CEUSEDMUX"
            lines.append(line)
        
        
        
def write_sitepips(tiles, tile, lines):
    for half in range(2):
        if len(tiles[tile][half]) == 0: continue
        site = tiles[tile][half][0]
        is_mtile = "CLBLM" in tile.getName()
        is_slicem = is_mtile and (half == 0)
        for sitepip in site.getUsedSitePIPs():
            if "SRUSEDMUX" in str(sitepip): continue
            if "CEUSEDMUX" in str(sitepip): continue
            if "CLKINV" in str(sitepip): continue
            if "USED" in str(sitepip): continue
            line = tile.getName() + "." + get_half_name(half,is_mtile) + "." + str(sitepip).split("-")[0]
            lines.append(line)


def write_logic(design):
    lines = []
    tiles = get_used_logic_tiles(design) 
    for tile in tiles.keys():
        write_luts_config(tiles, tile, lines)
        write_ff_config(tiles, tile, lines)
        write_sitepips(tiles, tile, lines)
        line = ""
        lines.append(line)
    return lines
    
    

def write_nets(design):
    lines = []
    for net in design.getNets():
        for pip in net.getPIPs():
            line = pip.getTile().getName() + "." + pip.getStartWire().getWireName() + "." + pip.getEndWire().getWireName()
            lines.append(line)
        line = ""
        lines.append(line)
    return lines

def write_clocking(design):
    # Notes: "16" and "12" for BUFGCTRL and BUFHCE respectively represent the maximum number of rows of primitives in a given tile (columns are always 2)
    # Hence, while getInstanceY() will give the global row_index, we needed the local row_index, which is why we subtract 16 or 12 from the row number
    lines = []
    vcc_nets = []
    pip_junctions = []
    
    for net in design.getNets():
        if net.isStaticNet() and net.getType() == NetType.VCC:
            vcc_nets.append(net)
            
    for cell in design.getCells():
        if "BUFGCTRL" in cell.getType(): 
            site = cell.getSite()
            line = cell.getTile().getName() + ".BUFGCTRL.BUFGCTRL_X0Y" + str(cell.getSiteInst().getSite().getInstanceY()-16) + ".IN_USE"
            lines.append(line)
            for bel in site.getBELs():
                for pin in bel.getPins():
                    if pin.isSitePort():
                        for vcc_net in vcc_nets:
                            for vcc_pin in vcc_net.getPins():
                                if pin == vcc_pin.getBELPin():
                                    line = cell.getTile().getName() + ".BUFGCTRL.BUFGCTRL_X0Y" + str(cell.getSiteInst().getSite().getInstanceY()-16) + ".ZINV_" + pin.getName()
                                    lines.append(line)
        
        if "BUFHCE" in cell.getType():
                site = cell.getSite()
                line = cell.getTile().getName() + ".BUFHCE.BUFHCE_X" + str(cell.getSiteInst().getSite().getInstanceX()) +"Y" + str(cell.getSiteInst().getSite().getInstanceY()%12) + ".IN_USE"
                lines.append(line)
                for bel in site.getBELs():
                    for pin in bel.getPins():
                        if pin.isSitePort():
                            for vcc_net in vcc_nets:
                                for vcc_pin in vcc_net.getPins():
                                    if pin == vcc_pin.getBELPin():
                                        line = cell.getTile().getName() + ".BUFHCE.BUFHCE_X" + str(cell.getSiteInst().getSite().getInstanceX()) + "Y" + str(cell.getSiteInst().getSite().getInstanceY()%12) + ".ZINV_" + pin.getName()
                                        if not line in lines:
                                            lines.append(line)
                for net in get_main_clock_nets(design):
                    for pip in net.getPIPs():
                        if not pip.getTile() == cell.getTile(): continue
                        line = pip.getTile().getName() + "." + pip.getStartWire().getWireName() + "_ACTIVE"
                        if not line in lines:
                            lines.append(line)
                for net in get_secondary_clock_nets(design):
                    for pip in net.getPIPs():
                        if (pip.getTile().getTileYCoordinate() == cell.getTile().getTileYCoordinate()) and not (pip.getTile() == cell.getTile()):
                            line = pip.getTile().getName() + ".ENABLE_BUFFER." + pip.getStartWire().getWireName()  
                            if not line in lines:
                                lines.append(line)
    
    return lines
        
        
 
 
def write_io(design):
    lines = []
    for cell in design.getCells():
        if "IBUF" in cell.getType() or "OBUF" in cell.getType() : 
            site = cell.getSiteInst()
            tile = cell.getTile()
            tile_name = tile.getName()
            half = 1-tile.getSiteIndex(cell.getSite())
            iostandard = cell.getProperty("IOSTANDARD").getValue()
            slew = cell.getProperty("SLEW").getValue()
            in_term = cell.getProperty("IN_TERM").getValue()
            pull_type = cell.getProperty("PULL_TYPE").getValue()
            if "IBUF" == cell.getType(): 
                if iostandard == "LVCMOS33" or iostandard == "LVTTL" or iostandard == "LVCMOS25":
                    line = tile_name + ".IOB_Y" + str(half) + ".LVCMOS25_LVCMOS33_LVTTL.IN"
                    lines.append(line)
                if iostandard == "SSTL135":
                    line = tile_name + ".IOB_Y" + str(half) + ".SSTL135.IN"
                    lines.append(line)
                    if not (in_term == "NONE"):
                        line = tile_name + ".IOB_Y" + str(half) + ".IN_TERM." + in_term
                        lines.append(line)           
                line = tile_name + ".IOB_Y" + str(half) + ".LVCMOS12_LVCMOS15_LVCMOS18_LVCMOS25_LVCMOS33_LVTTL_SSTL135.IN_ONLY"
                lines.append(line)                              
            if "OBUF" == cell.getType():
                if (iostandard == "LVCMOS33" or iostandard == "LVTTL"):
                    line = tile_name + ".IOB_Y" + str(half) + ".LVCMOS33_LVTTL.DRIVE.I12_I16"
                    lines.append(line)
                if (iostandard == "SSTL135"):
                    line = tile_name + ".IOB_Y" + str(half) + ".SSTL135.DRIVE.I_FIXED"
                    lines.append(line)
            if (slew == "SLOW"):
                line = tile_name + ".IOB_Y" + str(half) + ".LVCMOS12_LVCMOS15_LVCMOS18_LVCMOS25_LVCMOS33_LVTTL_SSTL135.SLEW.SLOW"
                lines.append(line)
            elif (iostandard == "SSTL135"):
                line = tile_name + ".IOB_Y" + str(half) + ".SSTL135.SLEW.FAST"
                lines.append(line)
            else:
                line = cell.getTile().getName() + ".IOB_Y" + str(half) + ".LVCMOS12_LVCMOS15_LVCMOS18_LVCMOS25_LVCMOS33_LVTTL.SLEW.FAST"
                lines.append(line)
            line = cell.getTile().getName() + ".IOB_Y" + str(half) + ".PULLTYPE." + pull_type
            lines.append(line)
            line = ""
            lines.append(line)
            
    return lines

def get_pseudo_pip_data():
    pp_config = dict()
    for s in ["L","R"]:
        for s2 in ["", "_TBYTESRC", "_TBYTETERM", "_SING"]:
            lst = ["","0","1"] if (s2 == "_SING") else ["0","1"]
            for i in lst:
                pp_config[(s + "IOI3" + s2, s + "IOI_OLOGIC" + i + "_OQ","IOI_OLOGIC" + i + "_D1")] = ["OLOGIC_Y" + i + ".OMUX.D1", "OLOGIC_Y" + i + ".OQUSED", "OLOGIC_Y" + i + ".OQUSED","OLOGIC_Y" + i + ".OSERDES.DATA_RATE_TQ.BUF"]
                pp_config[(s + "IOI3" + s2, "IOI_ILOGIC" + i + "_O", s + "IOI_ILOGIC" + i + "_D")] = ["IDELAY_Y" + i + ".IDELAY_TYPE_FIXED","ILOGIC_Y" + i + ".ZINV_D"]             
                pp_config[(s + "IOI3" + s2, s + "IOI_OLOGIC" + i + "_TQ", "IOI_OLOGIC" + i + "_T1")] = ["OLOGIC_Y" + i + ".ZINV_T1"]
                if (i == "0"): 
                    pp_config[(s + "IOB33" + s2, "IOB_O_IN1", "IOB_O_OUT0")] = []
                    pp_config[(s + "IOB33" + s2, "IOB_O_OUT0", "IOB_O0")] = []
                    pp_config[(s + "IOB33" + s2, "IOB_T_IN1", "IOB_T_OUT0")] = []
                    pp_config[(s + "IOB33" + s2, "IOB_T_OUT0", "IOB_T0")] = []

    for s1 in ["TOP", "BOT"]:
        for s2 in ["L", "R"]: 
            for i in range(12): 
                ii = str(i)
                hck = s2 + ii
                buf = ("X1Y" if (s2 == "R")  else "X0Y") + ii
                pp_config[("CLK_HROW_" + s1 + "_R", "CLK_HROW_CK_HCLK_OUT_" + hck,"CLK_HROW_CK_MUX_OUT_" + hck)] = ["BUFHCE.BUFHCE_" + buf + ".IN_USE","BUFHCE.BUFHCE_" + buf + ".ZINV_CE"]
        for i in range(16):
            ii = str(i)
            pp_config[("CLK_BUFG_" + s1 + "_R", "CLK_BUFG_BUFGCTRL" + ii + "_O","CLK_BUFG_BUFGCTRL" + ii + "_I0")] = ["BUFGCTRL.BUFGCTRL_X0Y" + ii + ".IN_USE", "BUFGCTRL.BUFGCTRL_X0Y" + ii + ".IS_IGNORE1_INVERTED","BUFGCTRL.BUFGCTRL_X0Y" + ii + ".ZINV_CE0", "BUFGCTRL.BUFGCTRL_X0Y" + ii + ".ZINV_S0"]
            pp_config[("CLK_BUFG_" + s1 + "_R", "CLK_BUFG_BUFGCTRL" + ii + "_O","CLK_BUFG_BUFGCTRL" + ii + "_I1")] = ["BUFGCTRL.BUFGCTRL_X0Y" + ii + ".IN_USE", "BUFGCTRL.BUFGCTRL_X0Y" + ii + ".IS_IGNORE0_INVERTED","BUFGCTRL.BUFGCTRL_X0Y" + ii + ".ZINV_CE1", "BUFGCTRL.BUFGCTRL_X0Y" + ii + ".ZINV_S1"]


    rclk_y_to_i = [2, 3, 0, 1]
    for y in range(4): 
        yy = str(y)
        ii = str(rclk_y_to_i[y])
        pp_config[("HCLK_IOI3", "HCLK_IOI_RCLK_OUT" + ii,"HCLK_IOI_RCLK_BEFORE_DIV" + ii)] = ["BUFR_Y" + yy + ".IN_USE","BUFR_Y" + yy + ".BUFR_DIVIDE.BYPASS"]


    for s in ["L", "R"]: 
        for i in range(24):
            ii = str(i)
    #        pp_config[("INT_INTERFACE_" + s, "INT_INTERFACE_LOGIC_OUTS_" + s + ii,"INT_INTERFACE_LOGIC_OUTS_" + s + "_B" + ii)]
            
    return pp_config



def write_pips(design):
    lines = []
    pp_config = get_pseudo_pip_data()
    for net in design.getNets():
        if str(net) == "<const0>": continue
        for pip_index in range(len(net.getPIPs())):
            pip = net.getPIPs()[pip_index]
            dst_intent = pip.getEndWire()
            if dst_intent.getNode().isTied():  continue
            tile_name = pip.getTile().getName()
            src = pip.getStartWire().getWireName()
            dst = pip.getEndWire().getWireName()
            if pip.isBidirectional() and (net.getPIPs()[pip_index+1].getStartWire() in pip.getTile().getWireConnections(src)):
                dst = pip.getStartWire().getWireName()
                src = pip.getEndWire().getWireName()
            ppk = (pip.getTile().getTileTypeEnum().toString() ,dst,src)
            if not (pp_config.get(ppk)):
                line = tile_name + "." + dst + "." + src
                lines.append(line)  
                continue
                
            if len(pp_config.get(ppk)):
                pp = pp_config.get(ppk)
                for c in pp:
                    line = tile_name + "." + c
                    lines.append(line)
            else:
                if ("DSP_L" in tile_name) or ("DSP_R" in tile_name): continue
                if ("RIOI3_SING" in tile_name) or ("LIOI3_SING" in tile_name): 
                    if (("IMUX" in src) or ("CTRL0" in src)) and (not ("CLK" in dst)): continue
                    if ("_SING_" in src):
                        src_split = src.split("_")
                        src = src_split[0] + src_split[2]

                line = tile_name + "." + dst + "." + src
                lines.append(line)
                if ("IOI3" in tile_name) and ("IOI_OCLK_" in dst):
                    dst_split = dst.split("_")
                    dst = ""
                    for i in range(len(dst_split)):
                        if i > 0:
                            dst = dst + "_"
                        dst = dst + dst_split[i]
                        if i == 1:
                            dst = dst + "M"
                    line = tile_name + "." + dst + "." + src
                    lines.append(line)  
        line = ""
        lines.append(line)
    return lines


def write_route_thru_pips(design):
    lines = []
    tracker = []
    bel_tracker = []
    for net in design.getNets():
        for j in range(len(net.getPIPs())):
            pip = net.getPIPs()[j]
            if pip.isRouteThru():
                wires = []
                wires.append(pip.getStartWire())
                wires.append(pip.getEndWire())
                for i in range(2):
                    wire = wires[i]
                    site = wire.getSitePin().getSite()
                    bel_pin = wire.getSitePin().getBELPin()
                    for site_conn in bel_pin.getSiteConns():
                        bel = site_conn.getBEL()
                        if "LUT" in bel.getBELType():
                            if not pip in tracker:
                                tracker.append(pip)
                                lut_name = bel_pin.getSiteWireName()[0] + "LUT"
                                pin_index = int(bel_pin.getSiteWireName()[1])
                                val = ""
                                for k in range(64):
                                    if (k>>(pin_index-1)&1):
                                        val = '1' + val
                                    else:
                                        val = '0' + val
                                line = str(site.getTile()) + "." + get_half_name(site.getSiteIndexInTile(),"CLBLM" in site.getTile().getName()) + "." + lut_name + ".INIT[63:0] = 64'b" + val
                                lines.append(line)
                                out_wires = []
                                for forward_pip in net.getPIPs()[j+1:]:
                                    if forward_pip.getTile() == pip.getTile():
                                        if (not forward_pip.getStartWire() in out_wires) :
                                            out_wires.append(forward_pip.getStartWire())
                                        if not forward_pip.getEndWire() in out_wires:
                                            out_wires.append(forward_pip.getEndWire())
                                for out_wire in out_wires:
                                    out_site_pin = out_wire.getSitePin()
                                    if out_site_pin:
                                        out_bel_pin = out_site_pin.getBELPin()
                                        for out_bel_pin_conn in out_bel_pin.getSiteConns():
                                            out_rbel_pins = out_bel_pin_conn.getBEL().getPins()                                            
                                            if len(out_rbel_pins) <= 2: continue
                                            for out_rbel_pin in out_rbel_pins:
                                                for out_rbel_pin_site_conn in out_rbel_pin.getSiteConns():
                                                    for lut_bel_site_pin in bel.getPins():
                                                        if out_rbel_pin_site_conn == lut_bel_site_pin:
                                                            if ((out_rbel_pin,site)) not in bel_tracker:
                                                                bel_tracker.append((out_rbel_pin,site))
                                                                line = str(site.getTile()) + "." + get_half_name(site.getSiteIndexInTile(),"CLBLM" in site.getTile().getName()) + "." + out_bel_pin_conn.getBEL().getName() + "." + out_rbel_pin.getName()
                                                                lines.append(line)
            line = ""
            lines.append(line)
    return lines


def write_fasm(design):
    lines = []
    lines = write_logic(design) 
    lines.extend(write_route_thru_pips(design))
    lines.extend(write_io(design))
    lines.extend(write_pips(design))
    lines.extend(write_clocking(design))

    f = open("top.fasm", "w")
    prev_line = ""
    for line in lines:
        if not ((prev_line == "") and (line == "")):
            f.write(line + """\n""")
        prev_line = line    
    f.close()


### run.py

In [21]:
import random
import os

def run_rapidwright(seed, topModule,device,filename,constraints):
    random.seed(seed)
    design = read_json(filename,topModule,device)
    net = design.getNetlist()
    net.setDevice(design.getDevice())
    topCell = net.getCell("top")

    placeIOBuffers(design , constraints)
    placeCells(design)
    createNets(design)
    createNetPins(design)
    routeSitePIPs(design)
    design.routeSites() # not necessary, but helps route input site wire for OBUFS
    routeClocks(design)
    try:
        designRouter(design)
        write_fasm(design)
        return 1
    except:
        return 0

### Utilities.py

In [11]:
import os

def run_yosys(topModule):
    yosys_cmd = """yosys -p "synth_xilinx -flatten -abc9 -nobram -arch xc7 -top """+topModule+"""; write_json """+topModule+""".json" """+topModule+""".v"""
    ret = os.system(yosys_cmd)
    

def generate_vivado_bistream(topModule):
    tcl_code="""
    open_checkpoint """+topModule+""".dcp
    write_bitstream -force """+topModule+"""
    """
    tcl_file = open(topModule+".tcl","w")
    ret = tcl_file.write(tcl_code)
    tcl_file.close()
    vivado_cmd = """vivado vivado_files/"""+topModule+""".xpr -nolog -nojournal -notrace -mode batch -source """+topModule+""".tcl"""
    ret = os.system(vivado_cmd)

### Main.py

In [24]:
topModule = "top"
device = "xc7a35tcsg324-1"
filename = "top.json"

constraints = {
               'i_clk' : {'LOC' : 'E3', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'FAST', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'in0[1]' : {'LOC' : 'C11', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'FAST', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'in0[0]' : {'LOC' : 'A8', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'FAST', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'in1[1]' : {'LOC' : 'A10', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'FAST', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'in1[0]' : {'LOC' : 'C10', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'FAST', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'out[3]' : {'LOC' : 'K1', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'FAST', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'out[2]' : {'LOC' : 'J3', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'FAST', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'out[1]' : {'LOC' : 'G3', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'FAST', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'out[0]' : {'LOC' : 'G6', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'FAST', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'}
                }

run_yosys(topModule)

for i in range(1000):
    if (run_rapidwright(i, topModule,device,filename,constraints)):
        print "Seed Value: " + str(i)
        break
        
print "Done"

Seed Value: 262
Done
